In [19]:
import numpy as np
import cv2, os
import matplotlib.pyplot as plt
import random, math
import glob
from PIL import Image

# Square map of
def run_all(MAP_SIZE = 300,AGENTS_NUM = 50000,BLUR_STRENTH = 3,DISSIPATE_TARGET = 200000,NUDGE_STRENGTH = 2,STEP_SIZE = 1,CIRCLE_SIZE = 0.4,ITERATIONS = 500,RENDER_SPEED = 1,TRAIL_STRENGTH = 1,SPEED = 1,LOOK_DISTANCE = 5,MAX_ACCELERATION = 0.5,OOB_NUDGE = 0.0,OOB_POWER = 0.0,texture_map = None,agents = None):
    def sign (i):
        if i >= 0:
            return 1
        elif i < 0:
            return -1

    def init_scene ():
        # Update circle size
        _circle = int(CIRCLE_SIZE * MAP_SIZE)

        # Give the agents some positions towards the middle
        agents = np.zeros((AGENTS_NUM, 4))
        for i in range(AGENTS_NUM):
            while True:
                x = random.randint(-_circle, _circle)
                y = random.randint(-_circle, _circle)
                if x ** 2 + y ** 2 > _circle ** 2:
                    continue
                agents[i, 0] = x + MAP_SIZE / 2
                agents[i, 1] = y + MAP_SIZE / 2
                agents[i, 2] = (random.random() - 0.5) * 10
                agents[i, 3] = (random.random() - 0.5) * 10
                
                break
        agents += np.random.rand(AGENTS_NUM, 4) * 0.1


        # Blurrs the map and then reduces the intensity of the map
    def dissipate_trails ( _map ) :
        _map += np.random.random((MAP_SIZE, MAP_SIZE))/4
        _blurred = cv2.GaussianBlur(_map,(BLUR_STRENTH,BLUR_STRENTH),0)
        _blurred[_blurred > 20] = 20
        while _blurred.sum() > DISSIPATE_TARGET:
            _blurred *= 0.95
        _map[:,:] = _blurred
        return _blurred

    # Moves the adgents along the map basted on their trails
    def update_agents ( _map, _agents ):

        # Compute the agents and the gradient
        integer_positions = np.rint(_agents[:,:2]).astype(int)
        integer_velocity = np.rint(_agents[:,2:]).astype(int)

        forward_look = (integer_velocity * LOOK_DISTANCE)

        left_look = np.zeros((AGENTS_NUM, 2))
        left_look[:,0] = forward_look[:,1]
        left_look[:,1] = -forward_look[:,0]

        right_look = np.zeros((AGENTS_NUM, 2))
        right_look[:,0] = -forward_look[:,1]
        right_look[:,1] = forward_look[:,0]

        forward_cast = (integer_positions + forward_look).astype(int)
        forward_cast[forward_cast > MAP_SIZE - 1] = MAP_SIZE - 1
        forward_cast[forward_cast < 0] = 0
        strength_forward = _map[forward_cast[:,0], forward_cast[:,1]]

        left_cast = (integer_positions + left_look).astype(int)
        left_cast[left_cast > MAP_SIZE - 1] = MAP_SIZE - 1
        left_cast[left_cast < 0] = 0
        strength_left = _map[left_cast[:,0], left_cast[:,1]]

        right_cast = (integer_positions + right_look).astype(int)
        right_cast[right_cast > MAP_SIZE - 1] = MAP_SIZE - 1
        right_cast[right_cast < 0] = 0
        strength_right = _map[right_cast[:,0], right_cast[:,1]]
        
        # Compute weighted average of the strengths
        total_strength = strength_forward + strength_left + strength_right

        percentage_forward = strength_forward / total_strength
        percentage_left = strength_left / total_strength
        percentage_right = strength_right / total_strength

        # Target_vel
        invert_vel = np.zeros((AGENTS_NUM, 2))
        invert_vel[:,0] = _agents[:,3]
        invert_vel[:,1] = -_agents[:,2]

        target_vel = np.zeros((AGENTS_NUM, 2))

        target_vel[:,0] += _agents[:,2] * percentage_forward
        target_vel[:,0] += invert_vel[:,0] * percentage_left
        target_vel[:,0] -= invert_vel[:,0] * percentage_right

        target_vel[:,1] += _agents[:,3] * percentage_forward
        target_vel[:,1] += invert_vel[:,1] * percentage_left
        target_vel[:,1] -= invert_vel[:,1] * percentage_right


        _agents[:,2:] += target_vel * STEP_SIZE * NUDGE_STRENGTH

        _velocity = np.sqrt(_agents[:,2] ** 2 + _agents[:,3] ** 2)
        _agents[:,2] *= SPEED/_velocity
        _agents[:,3] *= SPEED/_velocity

        _agents[:,0] += _agents[:,2] * STEP_SIZE
        _agents[:,1] += _agents[:,3] * STEP_SIZE

        # Find and correct out of bounds agents
        _x_oob = np.any([_agents[:,0] <= 0, _agents[:,0] >= MAP_SIZE-1], axis=0)
        _y_oob = np.any([_agents[:,1] <= 0, _agents[:,1] >= MAP_SIZE-1], axis=0)

        _agents[:,0] = np.clip(_agents[:,0], 0, MAP_SIZE-1)
        _agents[:,1] = np.clip(_agents[:,1], 0, MAP_SIZE-1)

        # Bounce off the walls
        _agents[:,2][_x_oob] *=  np.random.random((1))[0]/3 - 1.17
        _agents[:,3][_y_oob] *=  np.random.random((1))[0]/3 - 1.17

        # _agents[:,2][_agents[:,0] > 200] -= (_agents[:,0][_agents[:,0] > 200] - (MAP_SIZE/2))/400
        # _agents[:,3][_agents[:,1] > 200] -= (_agents[:,1][_agents[:,1] > 200] - (MAP_SIZE/2))/400
        # _agents[:,2][_agents[:,0] < 100] -= (_agents[:,0][_agents[:,0] < 100] - (MAP_SIZE/2))/400
        # _agents[:,3][_agents[:,1] < 100] -= (_agents[:,1][_agents[:,1] < 100] - (MAP_SIZE/2))/400
        

        # Find the agents that are out of bounds
        # oob = int(OOB_NUDGE * MAP_SIZE)
        # _oob = np.any([agents[:,0] < oob, agents[:,0] >= MAP_SIZE-oob, agents[:,1] < oob, agents[:,1] >= MAP_SIZE-oob], axis=0)

        # Nudge the agents back into the map
        # _agents[:,2][_oob] -= np.sign(_agents[:,0][_oob] - MAP_SIZE//2) * OOB_POWER
        # _agents[:,3][_oob] -= np.sign(_agents[:,1][_oob] - MAP_SIZE//2) * OOB_POWER

        # Scale to speed
        
        # Lays trails down on the map
    def lay_trails ( _map, _agents ):
        integer_positions = np.rint(_agents[:,:2]).astype(int)
        _map[integer_positions[:,0], integer_positions[:,1]] += TRAIL_STRENGTH


    # Highlights the agents on the map
    def tag_agents ( _map, _agents, offset):

        _intensity = 1
        integer_positions = np.rint(_agents[:,:2]).astype(int)
        integer_positions += offset
        
        # _map[integer_positions[:,0], integer_positions[:,1]] = _intensity
        
        # _map[integer_positions[:,0]-1, integer_positions[:,1]] = _intensity
        # _map[integer_positions[:,0], integer_positions[:,1]-1] = _intensity
        # _map[integer_positions[:,0]+1, integer_positions[:,1]] = _intensity
        # _map[integer_positions[:,0], integer_positions[:,1]+1] = _intensity

        # _map[integer_positions[:,0]-1, integer_positions[:,1]-1] = _intensity
        # _map[integer_positions[:,0]+1, integer_positions[:,1]-1] = _intensity
        # _map[integer_positions[:,0]+1, integer_positions[:,1]+1] = _intensity
        # _map[integer_positions[:,0]-1, integer_positions[:,1]+1] = _intensity

    # A single step
    def simulation_step (_map, _agents):
        dissipate_trails(_map)
        update_agents(_map, _agents)
        lay_trails(_map, _agents)

    # Renders simply to the screen
    def show_map ( m, save , do) :
        
        # copy the map to a new array and padd
        _map = np.zeros((MAP_SIZE+4, MAP_SIZE+4))
        _map[2:-2,2:-2] = m

        # Tag the agents
        #tag_agents(_map, agents, 2)

        # Scale the map
        _map = np.log10(_map)
        _map = _map / _map.max() * 255
        _map[0,0] = 255
        
        _colored = np.zeros((MAP_SIZE+4, MAP_SIZE+4, 3))
        _colored[:,:,1] = _map
        _colored[:,:,2] = (_map - 127) * 2
        _colored[:,:,0] = 0
        _colored[_colored < 0] = -_colored[_colored < 0]
        _colored[_colored > 255] = 255 -_colored[_colored > 255] 
        _colored[_colored < 0] = 0
        

        _colored = cv2.GaussianBlur(_colored,(BLUR_STRENTH,BLUR_STRENTH),0)

        _colored = _colored.astype(np.uint8)


        #print(_map.min(), _map.max())

        #plt.imshow(_map, cmap='gray')

        if not do: return

        # Render the map
        
        im = Image.fromarray(_colored)
        if im.mode != 'RGB':
            im = im.convert('RGB')
        im.save(f'results/{save[0]}/saves/{save[1]}.png')

    def run_simulation (MAP_SIZE = 300,AGENTS_NUM = 50000,BLUR_STRENTH = 3,DISSIPATE_TARGET = 200000,NUDGE_STRENGTH = 2,STEP_SIZE = 1,CIRCLE_SIZE = 0.4,ITERATIONS = 500,RENDER_SPEED = 1,TRAIL_STRENGTH = 1,SPEED = 1,LOOK_DISTANCE = 5,MAX_ACCELERATION = 0.5,OOB_NUDGE = 0.0,OOB_POWER = 0.0,texture_map = None,agents = None):

        # Setup Saving
        # global texture_map

        # Create the map
        texture_map = np.random.random((MAP_SIZE, MAP_SIZE))/100
        run_id = str(213)
        os.mkdir(f'results/')
        os.mkdir(f'results/saves')
        print(f'Saving as {run_id}')

        # Initialize the scene
        init_scene()

        # Run the simulation
        for i in range(ITERATIONS):
            print(f'{i+1}/{ITERATIONS}', end='\r')
            simulation_step(texture_map, agents)
            show_map(texture_map, (run_id, str.rjust(str(i),4,'0')), i % RENDER_SPEED == 0)
            # For fun
            # if i % 200 <= 0:
            #     texture_map = np.zeros((MAP_SIZE, MAP_SIZE))
            
        return run_id
    f = np.zeros((5,5))
    i = np.array([[4,4],[2,4]])

    f[i[:,0], i[:,1]]

    sim_id = run_simulation(MAP_SIZE = 300,AGENTS_NUM = 50000,BLUR_STRENTH = 3,DISSIPATE_TARGET = 200000,NUDGE_STRENGTH = 2,STEP_SIZE = 1,CIRCLE_SIZE = 0.4,ITERATIONS = 500,RENDER_SPEED = 1,TRAIL_STRENGTH = 1,SPEED = 1,LOOK_DISTANCE = 5,MAX_ACCELERATION = 0.5,OOB_NUDGE = 0.0,OOB_POWER = 0.0,texture_map = None,agents = None)

    def compile_results ( _id, MAP_SIZE = 300,AGENTS_NUM = 50000,BLUR_STRENTH = 3,DISSIPATE_TARGET = 200000,NUDGE_STRENGTH = 2,STEP_SIZE = 1,CIRCLE_SIZE = 0.4,ITERATIONS = 500,RENDER_SPEED = 1,TRAIL_STRENGTH = 1,SPEED = 1,LOOK_DISTANCE = 5,MAX_ACCELERATION = 0.5,OOB_NUDGE = 0.0,OOB_POWER = 0.0,texture_map = None,agents = None ):
        global texture_map
        global agents

        # Create the map
        texture_map = np.random.random((MAP_SIZE, MAP_SIZE))/100
        # Load all the images
        images = glob.glob(f"results/{sim_id}/saves/*.png")
        images.sort()

        frames = [Image.open(image) for image in images]
        frame_one = frames[0]
        frame_one.save(f"results/{_id}/{MAP_SIZE}/{AGENTS_NUM}/{BLUR_STRENTH}/{DISSIPATE_TARGET}/{NUDGE_STRENGTH}/out.gif", format="GIF", append_images=frames,
                save_all=True, duration=20, loop=0)
    # Create gif and show it
    compile_results( sim_id, MAP_SIZE = 300,AGENTS_NUM = 50000,BLUR_STRENTH = 3,DISSIPATE_TARGET = 200000,NUDGE_STRENGTH = 2,STEP_SIZE = 1,CIRCLE_SIZE = 0.4,ITERATIONS = 500,RENDER_SPEED = 1,TRAIL_STRENGTH = 1,SPEED = 1,LOOK_DISTANCE = 5,MAX_ACCELERATION = 0.5,OOB_NUDGE = 0.0,OOB_POWER = 0.0,texture_map = None,agents = None)
    

In [21]:
run_all(MAP_SIZE = 300,AGENTS_NUM = 50000,BLUR_STRENTH = 3,DISSIPATE_TARGET = 200000,NUDGE_STRENGTH = 2,STEP_SIZE = 1,CIRCLE_SIZE = 0.4,ITERATIONS = 500,RENDER_SPEED = 1,TRAIL_STRENGTH = 1,SPEED = 1,LOOK_DISTANCE = 5,MAX_ACCELERATION = 0.5,OOB_NUDGE = 0.0,OOB_POWER = 0.0,texture_map = None,agents = None)

Saving as 213


TypeError: 'NoneType' object is not subscriptable

In [14]:
ls

expt.ipynb  particles.ipynb  Pipfile  results/
